In [1]:
"""
Problem Statement

Build a model which predicts sales 
based on the money spent on different category
for online shopping mall
"""

'\nProblem Statement\n\nBuild a model which predicts sales \nbased on the money spent on different category\nfor online shopping mall\n'

In [2]:
# library
import pandas as pd
import numpy as np
import seaborn as sb

from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

from copy import deepcopy

from scipy.stats import norm, skew # for some statistics
from scipy import stats #qqplot
import statsmodels.api as sm # for decomposing the trends, seasonality etc.

from statsmodels.tsa.statespace.sarimax import SARIMAX # the big daddy

In [3]:
from enum import IntEnum

Column = IntEnum("Column", "DATE DEMAND CATEGORY CODE COLOR")
Dataset = IntEnum("Dataset", "ORIGIN ACCU_RATE ACCU_DATA")

In [8]:
# read original data file
data = pd.read_csv('dali_data.csv')

dataset = pd.DataFrame(
    columns=list(name for name, __ in Column.__members__.items()))

dataset[[Column.DATE.name, Column.CATEGORY.name, Column.DEMAND.name]] = data[
    ["date", "category", "quantity"]]

dataset[Column.DATE.name] = pd.to_datetime(dataset[Column.DATE.name])

# separate item ID and item color
dataset[Column.CODE.name] = pd.DataFrame(data['order_id'].str.split('-').tolist())[1]
dataset[Column.COLOR.name] = pd.DataFrame(data['order_item_code'].str.split('-').tolist())[2]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,4,14,18,20,22,23,24,26,27,28,29,32,33,35,44,49,53,54,55,56,74,76,77,79,80,81,82,83,86,87,89,93,94,95,98,102,109,110,111,112,116,119,120,124,125,126,127,128,129,130,131,132,133,137,138,139,140,142,144,145,146,150,151,153,155,157,159,160,161,162,163,164,166,170,172,173,174,176,177,178,179,180,181,182,183,184,187,188,189,190,192,194,198,202,204,205,206,208,209,210,211,212,213,214,215,216,217,221,222,225,227,228,229,230) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
def check_null(dataset):
    print('check any number of columns with NaN')
    print(dataset.isnull().any().sum(), ' / ', len(dataset.columns), '\n')

    print('check any number of data points with NaN')
    print(dataset.isnull().any(axis=1).sum(), ' / ', len(dataset), '\n')

    print('check where those nulls are')
    print(dataset.isna().sum(), '\n')
    print('Null to Dataset Ratio in Dates: ', dataset.isnull().sum()[0] / dataset.shape[0]*100)

In [10]:
# check the cardinality
print(dataset.shape)

# check the data types
print(dataset.dtypes)

(72659, 5)
DATE        datetime64[ns]
DEMAND               int64
CATEGORY            object
CODE                object
COLOR               object
dtype: object


In [11]:
check_null(dataset)

check any number of columns with NaN
1  /  5 

check any number of data points with NaN
96  /  72659 

check where those nulls are
DATE         0
DEMAND       0
CATEGORY    96
CODE         0
COLOR        0
dtype: int64 

Null to Dataset Ratio in Dates:  0.0


In [12]:
# get the lowest and highest dates in the dataset
dataset[Column.DATE.name].min(), dataset[Column.DATE.name].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2020-05-31 00:00:00'))

In [13]:
# modify dataset
dataset[Column.CATEGORY.name].replace({
    "원피스": "Dress", "블라우스": "Blouse", "스커트": "Skirt", "니트": "Knit", "티셔츠": "Tee",
    "귀걸이": "Earring", "바지": "Pants", "잠옷/홈웨어": "Homewear", "셔츠": "Shirt", "목걸이": "Necklace",
    "신발": "Shoes", "재킷": "Jacket", "숄/스카프/머플러": "Scarf", "코트": "Coat", "가디건": "Cardigan",
    "투피스": "Two piece", "기타소품": "Accessory", "양말": "Socks", "보정속옷": "Underwear",
    "기타상의": "Top", "기타여성잡화": "Accessory", "헤어엑세서리": "Accessory", "벨트": "Belts",
    "가방": "Bag", "스타킹": "Stockings", "반지": "Ring", "팔찌/발찌": "Accessory", "브라": "Underwear",
    "팬티": "Underwear", "점퍼": "Puffer jacket", "레깅스": "Leggings", "조끼": "Vest", "점프수트": "Jumpsuit"
}, inplace=True)

In [14]:
dataset[Column.CATEGORY.name].replace({
    "마스크": np.nan, "주얼리 세트": np.nan, "기타아우터": np.nan, "브로치": np.nan, "기타시계": np.nan,
    "장갑": np.nan, "휴대폰케이스": np.nan, "디지털시계": np.nan, "시계 액세서리": np.nan,
    "기타하의": np.nan, "기타용품": np.nan, "모자": np.nan
}, inplace=True)

In [15]:
# drop na's
dataset.dropna(axis=0, inplace=True) # remove all rows with na's
dataset.reset_index(drop=True, inplace=True)

In [16]:
check_null(dataset)

check any number of columns with NaN
0  /  5 

check any number of data points with NaN
0  /  72325 

check where those nulls are
DATE        0
DEMAND      0
CATEGORY    0
CODE        0
COLOR       0
dtype: int64 

Null to Dataset Ratio in Dates:  0.0


In [19]:
def reconstruct_by_date(dataset, category):
#     df_origin = dataset.groupby(
#         [Column.CATEGORY.name, Column.CODE.name, Column.COLOR.name, Column.DATE.name]).sum().reset_index()
    
    df_origin = dataset.groupby(
            [Column.CATEGORY.name, Column.DATE.name]).sum().reset_index()
    
    df_item = df_origin.loc[df_origin[Column.CATEGORY.name] == category].to_dict('records')
    
    df_new = pd.DataFrame(
        columns=list(name for name, __ in Column.__members__.items()))
    
    min_date = str(dataset[Column.DATE.name].min()).split(' ')[0]
    max_date = str(dataset[Column.DATE.name].max()).split(' ')[0]
    
    df_new[Column.DATE.name] = pd.date_range(min_date, max_date, freq='d')
    df_new[Column.CATEGORY.name] = category
    
    # treat as 0 where there is no demand
    df_new[Column.DEMAND.name].fillna(0, inplace=True)
#     df_new[Column.CODE.name].fillna('-', inplace=True)
#     df_new[Column.COLOR.name].fillna('-', inplace=True)
    
    for values in df_item:       
        df_new[Column.DEMAND.name].mask(
            df_new[Column.DATE.name] == values[Column.DATE.name], values[Column.DEMAND.name], inplace=True)
#         df_new[Column.CODE.name].mask(
#             df_new[Column.DATE.name] == values[Column.DATE.name], values[Column.CODE.name], inplace=True)
#         df_new[Column.COLOR.name].mask(
#             df_new[Column.DATE.name] == values[Column.DATE.name], values[Column.COLOR.name], inplace=True)
        
    return df_new.set_index(Column.DATE.name)

In [ ]:
def calculate_3days(df_category):
    # daily sales value for 3 days
    df_category['3days'] = df_category['order_demand'].rolling(3).sum()
    df_category['3days'] = df_category['3days'].fillna(0)
    
    # time series decomposition
    stl = sm.tsa.STL(df_category['3days']).fit()
    df_category['trend'] = list(stl.trend)
    
    check_null(df_category)
    
    rcParams['figure.figsize'] = 18, 8
    df_category['3days'].plot(
        title='3 Days ' + df_category.product_category[0] + ' Sales')
    df_category['trend'].plot()
    plt.show()

    return df_category

In [26]:
def data_preprocess(df, window):
    num = len(df) - window + 1

    dict_data = {i+1: list(df.iloc[i:num+i, Column.DEMAND-1]) for i in range(window)} # column: day 1 ~ day window
    df_accu_data = pd.DataFrame(
        np.add.accumulate(list(dict_data.values())).T, columns=[i+1 for i in range(window)])
    
    dict_data[Column.CATEGORY.name] \
        = df_accu_data[Column.CATEGORY.name] = list(df.iloc[0:num, Column.CATEGORY])
    
    df_origin = pd.DataFrame(dict_data)
    df_accu_rate = df_origin.copy(deep=True)

    df_origin['RATE'] = rating(df_origin, window)
    df_accu_rate['RATE'] = df_accu_data['RATE'] = rating(df_accu_data, window)
    
    return [df_origin, df_accu_rate, df_accu_data]

In [21]:
def rating(df, window):
    lst_rate = [
        (df[window][i] - df[1][i]) / abs(df[1][i])
        if abs(df[1][i]) != 0 else df[window][i] - df[1][i] # if day 1's demand is 0
        for i in range(len(df))
    ]
    
    return lst_rate

In [22]:
def data_process(dataset, window, rate):
    lst_category = sorted(list(dataset[Column.CATEGORY.name].unique()))
    df_dict = {category: reconstruct_by_date(dataset, category) for category in lst_category}
    
    dict_labeled_dataset = {}
     
    for category in lst_category:
        df_dict[category].reset_index(inplace=True)
        lst_preprocess = data_preprocess(df_dict[category], window)

        if category != lst_category[0]:
            for i in range(len(Dataset)):
                dict_labeled_dataset[i+1] = pd.concat(
                    [dict_labeled_dataset[i+1], lst_preprocess[i]])

        else:
            # create dataframe for the very first category
            dict_labeled_dataset[Dataset.ORIGIN] = lst_preprocess[Dataset.ORIGIN-1] # origin data + origin rate
            dict_labeled_dataset[Dataset.ACCU_RATE] = lst_preprocess[Dataset.ACCU_RATE-1] # origin data + accumulated rate
            dict_labeled_dataset[Dataset.ACCU_DATA] = lst_preprocess[Dataset.ACCU_DATA-1] # accumulated data + accumulated rate

    print("window")
    print(window)
    
    dict_labeled_dataset, lst_label_count = labeling(dict_labeled_dataset, rate)
    
    return dict_labeled_dataset, lst_label_count # origin label count, accumulated label count

In [23]:
def labeling(dict_labeled_dataset, rate):
    lst_label_count = []
    
    for i in range(len(Dataset)):
        dict_labeled_dataset[i+1].reset_index(drop=True, inplace=True)
        dict_labeled_dataset[i+1]['LABEL'] = np.where(
            dict_labeled_dataset[i+1]['RATE'].apply(lambda rate: rate > rate), 1, 0)
        
        print("\n rate description")
        print(dict_labeled_dataset[i+1]['RATE'].describe())

        label = np.array(dict_labeled_dataset[i+1]['LABEL'])
        label_count = np.where(label >= 1)[0].shape[0]
        lst_label_count.append(label_count)

        print("\ncount above rate " + str(rate))
        print(label_count)
    
    return dict_labeled_dataset, lst_label_count

In [24]:
def data_prepare(dataset):
    dict_dfs = {i+1: {} for i in range(len(Dataset))}
    dict_counts = deepcopy(dict_dfs)

    for i in range(len(Dataset)):
        dict_dfs[i+1] = {window: {} for window in range(min_window, max_window+1)}
        dict_counts[i+1] = {window: [] for window in range(min_window, max_window+1)}

    for window in range(min_window, max_window+1):
        for rate in range(min_rate, max_rate+1):
            dict_labeled_dataset, lst_label_count = data_process(dataset, window, rate)

            dict_dfs[Dataset.ORIGIN][window][rate] = dict_labeled_dataset[Dataset.ORIGIN]
            dict_dfs[Dataset.ACCU_RATE][window][rate] = dict_labeled_dataset[Dataset.ACCU_RATE]
            dict_dfs[Dataset.ACCU_DATA][window][rate] = dict_labeled_dataset[Dataset.ACCU_DATA]

            dict_counts[Dataset.ORIGIN][window].append(lst_label_count[Dataset.ORIGIN-1])
            dict_counts[Dataset.ACCU_RATE][window].append(lst_label_count[Dataset.ACCU_RATE-1])
            dict_counts[Dataset.ACCU_DATA][window].append(lst_label_count[Dataset.ACCU_DATA-1])
            
    return dict_dfs, dict_counts

In [ ]:
def show_graph(dict_counts):
    for i in range(len(Dataset)-1):
        df_count = pd.DataFrame(dict_counts[i]).T
        df_count.rename(columns=dict(zip(
            list(i for i in range(max_rate-min_rate+1)), list(rate for rate in range(
                min_rate, max_rate+1)))), inplace=True)
        
        plot_title = {ORIGIN: 'Original ', ACCU_RATE: 'Accumulated Rate ', ACCU_DATA: ' Accumulated Data '}
        
        ax = df_count.plot(title=plot_title[i] + 'Label Count', figsize=(18, 9))
        ax.set_xlabel('Window')
        ax.set_ylabel('Counts')
        
        print(df_count)
        print(df_count.describe())
        
        plt.show()

In [25]:
# main
min_window = 3
max_window = 3

min_rate = 3
max_rate = 4

dict_dfs, dict_counts = data_prepare(dataset)
show_graph(dict_counts)

window
3

 rate description
count    14420.000000
mean         0.254921
std          1.831077
min         -1.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        148.000000
Name: RATE, dtype: float64

count above rate 3
0

 rate description
count    14420.000000
mean         1.318778
std          2.998575
min          0.000000
25%          0.000000
50%          0.500000
75%          2.000000
max        242.000000
Name: RATE, dtype: float64

count above rate 3
0

 rate description
count    14420.000000
mean         1.318778
std          2.998575
min          0.000000
25%          0.000000
50%          0.500000
75%          2.000000
max        242.000000
Name: RATE, dtype: float64

count above rate 3
0
window
3

 rate description
count    14420.000000
mean         0.254921
std          1.831077
min         -1.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        148.000000
Name: RATE, dtype: float64

count above rate 4
0

 rate